# Problem 5 Jupyter Helper

# Building a Trie in Python

Before we start let us reiterate the key components of a Trie or Prefix Tree. A trie is a tree-like data structure that stores a dynamic set of strings. Tries are commonly used to facilitate operations like predictive text or autocomplete features on mobile phones or web search.

Before we move into the autocomplete function we need to create a working trie for storing strings.  We will create two classes:
* A `Trie` class that contains the root node (empty string)
* A `TrieNode` class that exposes the general functionality of the Trie, like inserting a word or finding the node which represents a prefix.

Give it a try by implementing the `TrieNode` and `Trie` classes below!

In [1]:
## Represents a single node in the Trie
class TrieNode:
    def __init__(self):
        ## Initialize this node in the Trie
        self.is_word = False
        self.children = {}
    
    def insert_char(self, char):
        ## Add a child node in this Trie
        assert type(char) == str, f'''Inserted character must be string!'''
        assert len(char) <= 1, f'''
        The length of the character must be less than or equal to 1!
        '''
        self.children[char] = TrieNode()

In [2]:
## The Trie itself containing the root node and insert/find functions
class Trie:
    def __init__(self):
        ## Initialize this Trie (add a root node)
        self.root = TrieNode()

    def insert_word(self, word):
        ## Add a word to the Trie
        current_node = self.root
        
        word = word.lower()
        for char in word:
            if char not in current_node.children:
                current_node.insert_char(char)
            current_node = current_node.children[char]
        
        current_node.is_word = True

    def find(self, prefix):
        ## Find the Trie node that represents this prefix
        current_node = self.root
        
        prefix = prefix.lower()
        for char in prefix:
            if char not in current_node.children:
                return None
            current_node = current_node.children[char]
        
        return current_node

# Finding Suffixes

Now that we have a functioning Trie, we need to add the ability to list suffixes to implement our autocomplete feature.  To do that, we need to implement a new function on the `TrieNode` object that will return all complete word suffixes that exist below it in the trie.  For example, if our Trie contains the words `["fun", "function", "factory"]` and we ask for suffixes from the `f` node, we would expect to receive `["un", "unction", "actory"]` back from `node.suffixes()`.

Using the code you wrote for the `TrieNode` above, try to add the suffixes function below. (Hint: recurse down the trie, collecting suffixes as you go.)

In [3]:
class TrieNode:
    def __init__(self):
        ## Initialize this node in the Trie
        self.is_word = False
        self.children = {}
    
    def insert_char(self, char):
        ## Add a child node in this Trie
        assert type(char) == str, f'''Inserted character must be string!'''
        assert len(char) <= 1, f'''
        The length of the character must be less than or equal to 1!
        '''
        char = char.lower()
        self.children[char] = TrieNode()
    
    def has_children(self):
        return len(self.children) > 0
    
    def recurse_suffix_helper(self, node = None, suffix = "",
                              suffix_set = set()):
        suffix_tracker = suffix_set
        
        if node == None:
            if not self.has_children():
                return suffix_tracker
        
        if node != None:
            if node.is_word:
                suffix_tracker.add(suffix)
            
        if node != None:
            if not node.has_children():
                return suffix_tracker
    
        output_collection = set()
        if node == None:
            char_dict = self.children
        else:
            char_dict = node.children
        for char in char_dict:
            new_suffix = suffix + char
            if node is None:
                new_node = self.children[char]
            else:
                new_node = node.children[char]
            output = self.recurse_suffix_helper(new_node, new_suffix,
                                                suffix_tracker)
            if len(output) >= 1:
                for i in output:
                    output_collection.add(i)
        return output_collection
        
    def suffixes(self):
        ## Recursive function that collects the suffix for 
        ## all complete words below this point
        output = self.recurse_suffix_helper()
        output = list(output)
        output.sort()
        return output

## DO NOT RUN THE CELL BELOW

In [7]:
different_tree = Trie()
word_list = ['stop', "stupid", "stoner", "stud", "stoned", "stupider"]

for word in word_list:
    different_tree.insert_word(word)

In [8]:
find_sto = different_tree.find("sto")

In [9]:
find_sto.suffixes()

['actory',
 'ction',
 'ion',
 'n',
 'nction',
 'ned',
 'ner',
 'on',
 'p',
 'tion',
 'un',
 'unction']

# Testing it all out

Run the following code to add some words to your trie and then use the interactive search box to see what your code returns.

In [4]:
MyTrie = Trie()
wordList = [
    "ant", "anthology", "antagonist", "antonym", 
    "fun", "function", "factory", 
    "trie", "trigger", "trigonometry", "tripod"
]
for word in wordList:
    MyTrie.insert_word(word)

In [5]:
from ipywidgets import widgets
from IPython.display import display
from ipywidgets import interact
def f(prefix):
    if prefix != '':
        prefixNode = MyTrie.find(prefix)
        if prefixNode:
            print('\n'.join(prefixNode.suffixes()))
        else:
            print(prefix + " not found")
    else:
        print('')
interact(f,prefix='');

interactive(children=(Text(value='', description='prefix'), Output()), _dom_classes=('widget-interact',))

In [6]:
# REFERENCES
# 1. Udacity Data Structures & Algorithms Nanodegree; 3) Basic Algorithms; 
#        1) Basic Algorithms; 6) Tries
# 2. Udacity Data Structures & Algorithms Nanodegree; 3) Basic Algorithms; 
#        4) Problems Vs Algorithms; 6) Problem 5: Autocomplete with Tries